In [37]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('sofr_futures_analysis.csv', index_col='date', parse_dates=True)

# Sort columns by contract expiry
df = df.sort_index(axis=1)

# Define packs based on available contracts
packs = ['1y', '2y', '3y', '4y']
pack_contracts = [df.columns[i:i+1] for i in range(0, len(df.columns)-1, 1)]  # Each contract is its own "pack"

# Calculate pack rates
for i, pack in enumerate(packs):
    if i < len(pack_contracts):
        df[f'{pack}_SOFR'] = df[pack_contracts[i]].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    df[f'{packs[i]}_{packs[i+1]}_diff'] = df[f'{packs[i]}_SOFR'] - df[f'{packs[i+1]}_SOFR']

# Calculate forward rates
df['1y1y_SOFR'] = (2 * df['2y_SOFR'] - df['1y_SOFR'])
df['2y1y_SOFR'] = (3 * df['3y_SOFR'] - 2 * df['2y_SOFR'])
df['3y1y_SOFR'] = (4 * df['4y_SOFR'] - 3 * df['3y_SOFR'])

# Save the results
df.to_csv('sofr_futures_analysis_refined.csv')

print("Analysis complete. Data saved to 'sofr_futures_analysis_refined.csv'")
print("\nSample of the data:")
print(df.tail())

# Plot the SOFR curve
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
for col in [col for col in df.columns if 'SOFR' in col and 'diff' not in col]:
    plt.plot(df.index, df[col], label=col)
plt.title('SOFR Curve')
plt.xlabel('Date')
plt.ylabel('Rate')
plt.legend()
plt.grid(True)
plt.savefig('sofr_curve.png')
plt.close()

print("\nSOFR curve plot saved as 'sofr_curve.png'")

Analysis complete. Data saved to 'sofr_futures_analysis_refined.csv'

Sample of the data:
             SFRG5   SFRG6   SFRG7   SFRQ4  near_pack  1y_SOFR  2y_SOFR  \
date                                                                      
2024-06-28  0.0710  0.0585  0.0475  0.0625   0.059000   0.0710   0.0585   
2024-07-01  0.0785  0.0675  0.0550  0.0675   0.067000   0.0785   0.0675   
2024-07-02  0.0775  0.0710  0.0600  0.0675   0.069500   0.0775   0.0710   
2024-07-03  0.0775  0.0725  0.0610  0.0675   0.070333   0.0775   0.0725   
2024-07-05  0.0785  0.0735  0.0610  0.0675   0.071000   0.0785   0.0735   

            3y_SOFR  4y_SOFR  1y_2y_diff  2y_3y_diff  3y_4y_diff  1y1y_SOFR  \
date                                                                          
2024-06-28   0.0475   0.0625      0.0125      0.0110     -0.0150     0.0460   
2024-07-01   0.0550   0.0675      0.0110      0.0125     -0.0125     0.0565   
2024-07-02   0.0600   0.0675      0.0065      0.0110     -0.0075    

In [99]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import io
import base64
from datetime import datetime

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Define chart titles and data
chart_data = [
    ('SOFR Implied Rates for Single Contracts', [col for col in pivot.columns if 'SR3' in col and '_' not in col]),
    ('SOFR Pack Rates', [col for col in pivot.columns if '_pack' in col]),
    ('SOFR Pack Differentials', [col for col in pivot.columns if '_diff' in col and '6m' not in col and 'spread' not in col]),
    ('SOFR 3-Month Calendar Spreads', [col for col in pivot.columns if '_spread' in col and '6m' not in col and '_diff' not in col]),
    ('SOFR 3-Month Calendar Spread Differentials', [col for col in pivot.columns if '_spread_' in col and '6m' not in col and '_diff' in col]),
    ('SOFR 6-Month Calendar Spreads', [col for col in pivot.columns if '6m_spread' in col and '6m_diff' not in col]),
    ('SOFR 6-Month Calendar Spread Differentials', [col for col in pivot.columns if '6m_diff' in col])
]

# Create Matplotlib figures
plt.figure(figsize=(20, len(chart_data) * 8))

for i, (title, cols) in enumerate(chart_data, 1):
    ax = plt.subplot(len(chart_data), 1, i)
    for col in cols:
        ax.plot(pivot.index, pivot[col], label=col)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Rate (bps)', fontsize=12)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()

# Save Matplotlib figure to a bytes buffer
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight', dpi=300)
plt.close()
buf.seek(0)
string = base64.b64encode(buf.read()).decode('utf-8')
html_string = f'<img src="data:image/png;base64,{string}">'

# Create Plotly figure
fig = make_subplots(rows=len(chart_data), cols=1, subplot_titles=[title for title, _ in chart_data])

for i, (title, cols) in enumerate(chart_data, start=1):
    for col in cols:
        fig.add_trace(
            go.Scatter(x=pivot.index, y=pivot[col], name=col, mode='lines'),
            row=i, col=1
        )
    fig.update_xaxes(title_text="Date", row=i, col=1)
    fig.update_yaxes(title_text="Rate (bps)", row=i, col=1)

fig.update_layout(height=300*len(chart_data), width=1200, title_text="SOFR Futures Analysis (Interactive)")




def prepare_yield_curve_data(pivot):
    contract_cols = [col for col in pivot.columns if 'SR3' in col and '_' not in col]
    # Sort the contract columns to ensure they're in the correct order
    contract_cols.sort(key=lambda x: (x[-4:], {'H': 1, 'M': 2, 'U': 3, 'Z': 4}[x[-5]]))
    yield_curve_data = pivot[contract_cols]
    return yield_curve_data, contract_cols

def plot_yield_curve(yield_curve_data, contract_cols, selected_dates=None):
    fig = go.Figure()

    if selected_dates is None:
        dates_to_plot = [
            yield_curve_data.index[-1],  # Most recent date
            yield_curve_data.index[-30],  # Approximately 1 month ago
            yield_curve_data.index[-90],  # Approximately 3 months ago
            yield_curve_data.index[-180],  # Approximately 6 months ago
            yield_curve_data.index[-365]  # Approximately 1 year ago
        ]
    else:
        dates_to_plot = selected_dates

    for plot_date in dates_to_plot:
        if plot_date in yield_curve_data.index:
            data = yield_curve_data.loc[plot_date].dropna()
            fig.add_trace(go.Scatter(
                x=list(range(len(data))),
                y=data,
                mode='lines+markers',
                name=plot_date.strftime('%Y-%m-%d'),
                text=data.index,
                hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
            ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis=dict(
            title='Contract Sequence',
            tickmode='array',
            tickvals=list(range(len(contract_cols))),
            ticktext=contract_cols,
            tickangle=45
        ),
        yaxis_title='Implied SOFR Rate (bps)',
        legend_title='Observation Date',
        hovermode='x unified'
    )

    return fig

# In your main script:
yield_curve_data, contract_cols = prepare_yield_curve_data(pivot)

if yield_curve_data is not None and len(contract_cols) > 0:
    # Option to select custom dates
    custom_dates = [
        datetime(2024, 7, 6),
        datetime(2024, 6, 6),
        datetime(2024, 4, 6),
        datetime(2024, 1, 6),
        datetime(2023, 7, 6)
    ]
    
    yield_curve_fig = plot_yield_curve(yield_curve_data, contract_cols, selected_dates=custom_dates)
    
    # Save the figure as HTML
    yield_curve_html = yield_curve_fig.to_html(full_html=False)
else:
    print("Unable to create yield curve plot due to data preparation issues")
    yield_curve_html = "<p>Unable to create SOFR Yield Curve Comparison due to data issues.</p>"

# Update the combined HTML to include the correct yield curve chart
combined_html = f"""
<html>
<head>
    <title>SOFR Futures Analysis</title>
</head>
<body>
    <h1>SOFR Futures Analysis</h1>
    <h2>Static Overview (High Resolution)</h2>
    {html_string}
    <h2>Interactive Charts</h2>
    {fig.to_html(full_html=False)}
    <h2>SOFR Yield Curve Comparison</h2>
    {yield_curve_html}
</body>
</html>
"""

# Save the combined HTML
with open(os.path.join(folder_path, 'sofr_analysis_dashboard_combined.html'), 'w') as f:
    f.write(combined_html)

print("Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'")

Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'


In [107]:
pip install dash dash-bootstrap-components

     ---------------------------------------- 7.5/7.5 MB 3.0 MB/s eta 0:00:00
     -------------------------------------- 222.5/222.5 kB 3.4 MB/s eta 0:00:00
  Using cached retrying-1.3.4-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [106]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import io
import base64
from datetime import datetime
import json
from dash import Dash, dcc, html, Input, Output
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

# ... [rest of your existing code for other charts] ...

def prepare_yield_curve_data(pivot):
    contract_cols = [col for col in pivot.columns if 'SR3' in col and '_' not in col]
    contract_cols.sort(key=lambda x: (x[-4:], {'H': 1, 'M': 2, 'U': 3, 'Z': 4}[x[-5]]))
    yield_curve_data = pivot[contract_cols]
    return yield_curve_data, contract_cols

def plot_yield_curve(yield_curve_data, contract_cols):
    fig = go.Figure()

    # Create a trace for each date
    for date in yield_curve_data.index:
        data = yield_curve_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=list(range(len(data))),
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>',
            visible=False  # Start with all traces hidden
        ))

    # Make the 5 most recent dates visible by default
    for trace in fig.data[-5:]:
        trace.visible = True

    # Create buttons for date selection
    buttons = []
    for i, date in enumerate(yield_curve_data.index):
        button = dict(
            args=[{'visible': [False] * len(fig.data)},
                  {'title': f'SOFR Yield Curve - Custom Selection'}],
            label=date.strftime('%Y-%m-%d'),
            method='update'
        )
        buttons.append(button)

    # Add 'Select All' and 'Deselect All' buttons
    buttons.append(dict(
        args=[{'visible': [True] * len(fig.data)},
              {'title': 'SOFR Yield Curve - All Dates'}],
        label='Select All',
        method='update'
    ))
    buttons.append(dict(
        args=[{'visible': [False] * len(fig.data)},
              {'title': 'SOFR Yield Curve - No Selection'}],
        label='Deselect All',
        method='update'
    ))

    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                x=0.0,
                y=1.15,
                showactive=True,
                buttons=buttons
            )
        ],
        xaxis=dict(
            title='Contract Sequence',
            tickmode='array',
            tickvals=list(range(len(contract_cols))),
            ticktext=contract_cols,
            tickangle=45
        ),
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig


# Prepare yield curve data and create plot
yield_curve_data, contract_cols = prepare_yield_curve_data(pivot)

if yield_curve_data is not None and len(contract_cols) > 0:
    yield_curve_fig = plot_yield_curve(yield_curve_data, contract_cols)
    yield_curve_html = yield_curve_fig.to_html(full_html=False, include_plotlyjs='cdn')
    print("Yield curve plot created successfully.")
else:
    print("Unable to create yield curve plot due to insufficient data")
    yield_curve_html = "<p>Unable to create SOFR Yield Curve Comparison due to insufficient data.</p>"

# Add JavaScript for multi-select functionality
js_code = """
<script>
document.addEventListener('DOMContentLoaded', (event) => {
    var gd = document.getElementById('yield-curve-plot');
    var buttons = gd.querySelectorAll('.modebar-btn');
    buttons.forEach(function(btn) {
        btn.addEventListener('click', function() {
            var visible = gd.data.map(trace => trace.visible);
            var visibleCount = visible.filter(Boolean).length;
            if (visibleCount > 5) {
                var lastVisible = visible.lastIndexOf(true);
                gd.data[lastVisible].visible = false;
                Plotly.redraw(gd);
            }
        });
    });
});
</script>
"""

# Combine all outputs into the HTML
combined_html = f"""
<html>
<head>
    <title>SOFR Futures Analysis</title>
</head>
<body>
    <h1>SOFR Futures Analysis</h1>
    <h2>Static Overview (High Resolution)</h2>
    {html_string}
    <h2>Interactive Charts</h2>
    {fig.to_html(full_html=False)}
    <h2>SOFR Yield Curve Comparison</h2>
    <div id="yield-curve-plot">
        {yield_curve_html}
    </div>
    {js_code}
</body>
</html>
"""

# Save the combined HTML
with open(os.path.join(folder_path, 'sofr_analysis_dashboard_combined.html'), 'w') as f:
    f.write(combined_html)

print("Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'")

Yield curve plot created successfully.
Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'


In [120]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import io
import base64
from dash import Dash, dcc, html, Input, Output, callback
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month butterfly spreads
butterfly_spreads = {}
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    spread_name = f'{front}_{middle}_{back}_butterfly'
    butterfly_spreads[spread_name] = pivot[front] - (2 * pivot[middle]) + pivot[back]

# Add butterfly spreads to pivot table
for name, spread in butterfly_spreads.items():
    pivot[name] = spread
    
# Prepare data for butterfly spread curve
butterfly_curve_data = pd.DataFrame(index=pivot.index)
for spread_name, spread_values in butterfly_spreads.items():
    middle_contract = spread_name.split('_')[1]
    butterfly_curve_data[middle_contract] = spread_values
    
# Create Dash app for butterfly spread curve
butterfly_app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

butterfly_app.layout = html.Div([
    html.H1("6-Month Butterfly Spread Curve"),
    dcc.DatePickerRange(
        id='butterfly-date-picker-range',
        min_date_allowed=butterfly_curve_data.index.min(),
        max_date_allowed=butterfly_curve_data.index.max(),
        initial_visible_month=butterfly_curve_data.index.max(),
        start_date=butterfly_curve_data.index[-5],
        end_date=butterfly_curve_data.index[-1]
    ),
    html.Button('Update Graph', id='butterfly-submit-button', n_clicks=0),
    dcc.Graph(id='butterfly-curve-graph')
])

@callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    if n_clicks == 0:
        raise PreventUpdate

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (butterfly_curve_data.index >= start_date) & (butterfly_curve_data.index <= end_date)
    selected_data = butterfly_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=data.index,
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Middle Contract',
            tickmode='array',
            tickvals=list(range(len(data.index))),
            ticktext=data.index,
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig

# Create scatterplots for butterfly spreads
def create_butterfly_scatterplots(pivot, butterfly_spreads):
    figs = []
    for spread_name, spread_values in butterfly_spreads.items():
        middle_contract = spread_name.split('_')[1]
        
        fig = go.Figure()
        
        x = pivot[middle_contract]
        y = spread_values
        
        # Sort the data by date
        sorted_indices = np.argsort(pivot.index)
        x = x.iloc[sorted_indices]
        y = y.iloc[sorted_indices]
        dates = pivot.index[sorted_indices]
        
        # Define color for each point
        colors = ['blue'] * len(x)
        if len(colors) > 0:
            colors[-1] = 'red'  # Latest value
        if len(colors) > 1:
            colors[-11:-1] = ['green'] * min(10, len(colors)-1)  # Next past 10 days
        if len(colors) > 11:
            colors[-41:-11] = ['orange'] * min(30, len(colors)-11)  # Next past 30 days
        
        # Add scatter plot
        fig.add_trace(go.Scatter(
            x=x, 
            y=y, 
            mode='markers',
            marker=dict(color=colors, size=8),
            text=dates,  # This will be shown on hover
            hovertemplate='Date: %{text}<br>X: %{x}<br>Y: %{y}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'Butterfly Spread: {spread_name}',
            xaxis_title=f'{middle_contract} Implied Rate',
            yaxis_title='Butterfly Spread Value',
            showlegend=False
        )
        
        figs.append(fig)
    
    return figs

butterfly_scatterplots = create_butterfly_scatterplots(pivot, butterfly_spreads)

# Define chart titles and data
chart_data = [
    ('SOFR Implied Rates for Single Contracts', [col for col in pivot.columns if 'SR3' in col and '_' not in col]),
    ('SOFR Pack Rates', [col for col in pivot.columns if '_pack' in col]),
    ('SOFR Pack Differentials', [col for col in pivot.columns if '_diff' in col and '6m' not in col and 'spread' not in col]),
    ('SOFR 3-Month Calendar Spreads', [col for col in pivot.columns if '_spread' in col and '6m' not in col and '_diff' not in col]),
    ('SOFR 3-Month Calendar Spread Differentials', [col for col in pivot.columns if '_spread_' in col and '6m' not in col and '_diff' in col]),
    ('SOFR 6-Month Calendar Spreads', [col for col in pivot.columns if '6m_spread' in col and '6m_diff' not in col]),
    ('SOFR 6-Month Calendar Spread Differentials', [col for col in pivot.columns if '6m_diff' in col]),
    ('SOFR 6-month Butterfly Spreads', list(butterfly_spreads.keys()))
]

# Create Matplotlib figures
plt.figure(figsize=(20, len(chart_data) * 8))

for i, (title, cols) in enumerate(chart_data, 1):
    ax = plt.subplot(len(chart_data), 1, i)
    for col in cols:
        ax.plot(pivot.index, pivot[col], label=col)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Rate (bps)', fontsize=12)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()

# Save Matplotlib figure to a bytes buffer
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight', dpi=300)
plt.close()
buf.seek(0)
string = base64.b64encode(buf.read()).decode('utf-8')
html_string = f'<img src="data:image/png;base64,{string}">'

# Create Plotly figure
fig = make_subplots(rows=len(chart_data), cols=1, subplot_titles=[title for title, _ in chart_data])

for i, (title, cols) in enumerate(chart_data, start=1):
    for col in cols:
        fig.add_trace(
            go.Scatter(x=pivot.index, y=pivot[col], name=col, mode='lines'),
            row=i, col=1
        )
    fig.update_xaxes(title_text="Date", row=i, col=1)
    fig.update_yaxes(title_text="Rate (bps)", row=i, col=1)

fig.update_layout(height=400*len(chart_data), width=1200, title_text="SOFR Futures Analysis (Interactive)")

# Prepare yield curve data
def prepare_yield_curve_data(pivot):
    contract_cols = [col for col in pivot.columns if 'SR3' in col and '_' not in col]
    contract_cols.sort(key=lambda x: (x[-4:], {'H': 1, 'M': 2, 'U': 3, 'Z': 4}[x[-5]]))
    yield_curve_data = pivot[contract_cols]
    return yield_curve_data, contract_cols

yield_curve_data, contract_cols = prepare_yield_curve_data(pivot)

# Create Dash app for yield curve comparison
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("SOFR Yield Curve Comparison"),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=yield_curve_data.index.min(),
        max_date_allowed=yield_curve_data.index.max(),
        initial_visible_month=yield_curve_data.index.max(),
        start_date=yield_curve_data.index[-5],
        end_date=yield_curve_data.index[-1]
    ),
    html.Button('Update Graph', id='submit-button', n_clicks=0),
    dcc.Graph(id='yield-curve-graph')
])

@app.callback(
    Output('yield-curve-graph', 'figure'),
    Input('submit-button', 'n_clicks'),
    Input('date-picker-range', 'start_date'),
    Input('date-picker-range', 'end_date')
)
def update_graph(n_clicks, start_date, end_date):
    if n_clicks == 0:
        raise PreventUpdate

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (yield_curve_data.index >= start_date) & (yield_curve_data.index <= end_date)
    selected_data = yield_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=list(range(len(data))),
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis=dict(
            title='Contract Sequence',
            tickmode='array',
            tickvals=list(range(len(contract_cols))),
            ticktext=contract_cols,
            tickangle=45
        ),
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

butterfly_scatterplots_html = "".join([fig.to_html(full_html=False, include_plotlyjs='cdn') for fig in butterfly_scatterplots])

# Modify the combined HTML to include the new butterfly spread curve
combined_html = f"""
<html>
<head>
    <title>SOFR Futures Analysis</title>
</head>
<body>
    <h1>SOFR Futures Analysis</h1>
    <h2>Static Overview (High Resolution)</h2>
    {html_string}
    <h2>Interactive Charts</h2>
    {fig.to_html(full_html=False)}
    <h2>Butterfly Spread Scatterplots</h2>
    {butterfly_scatterplots_html}
    <h2>6-Month Butterfly Spread Curve</h2>
    <iframe src="http://127.0.0.1:8051" width="100%" height="800px" frameborder="0"></iframe>
    <h2>SOFR Yield Curve Comparison</h2>
    <iframe src="http://127.0.0.1:8050" width="100%" height="800px" frameborder="0"></iframe>
</body>
</html>
"""

# Save the combined HTML
with open(os.path.join(folder_path, 'sofr_analysis_dashboard_combined.html'), 'w') as f:
    f.write(combined_html)

print("Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'")

# Run both Dash apps
if __name__ == '__main__':
    from threading import Thread
    Thread(target=lambda: butterfly_app.run_server(debug=True, port=8051)).start()
    app.run_server(debug=True, port=8050)

Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'


AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import io
import base64
from dash import Dash, dcc, html, Input, Output, callback
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from flask import Flask

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month butterfly spreads
butterfly_spreads = {}
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    spread_name = f'{front}_{middle}_{back}_butterfly'
    butterfly_spreads[spread_name] = pivot[front] - (2 * pivot[middle]) + pivot[back]

# Add butterfly spreads to pivot table
for name, spread in butterfly_spreads.items():
    pivot[name] = spread
    
# Prepare data for butterfly spread curve
butterfly_curve_data = pd.DataFrame(index=pivot.index)
for spread_name, spread_values in butterfly_spreads.items():
    middle_contract = spread_name.split('_')[1]
    butterfly_curve_data[middle_contract] = spread_values
    
# Create Dash app for butterfly spread curve
butterfly_app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

butterfly_app.layout = html.Div([
    html.H1("6-Month Butterfly Spread Curve"),
    dcc.DatePickerRange(
        id='butterfly-date-picker-range',
        min_date_allowed=butterfly_curve_data.index.min(),
        max_date_allowed=butterfly_curve_data.index.max(),
        initial_visible_month=butterfly_curve_data.index.max(),
        start_date=butterfly_curve_data.index[-5],
        end_date=butterfly_curve_data.index[-1]
    ),
    html.Button('Update Graph', id='butterfly-submit-button', n_clicks=0),
    dcc.Graph(id='butterfly-curve-graph')
])

@butterfly_app.callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    if n_clicks == 0:
        raise PreventUpdate

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (butterfly_curve_data.index >= start_date) & (butterfly_curve_data.index <= end_date)
    selected_data = butterfly_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=data.index,
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Middle Contract',
            tickmode='array',
            tickvals=list(range(len(data.index))),
            ticktext=data.index,
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig

# Create scatterplots for butterfly spreads
def create_butterfly_scatterplots(pivot, butterfly_spreads):
    figs = []
    for spread_name, spread_values in butterfly_spreads.items():
        middle_contract = spread_name.split('_')[1]
        
        fig = go.Figure()
        
        x = pivot[middle_contract]
        y = spread_values
        
        # Sort the data by date
        sorted_indices = np.argsort(pivot.index)
        x = x.iloc[sorted_indices]
        y = y.iloc[sorted_indices]
        dates = pivot.index[sorted_indices]
        
        # Define color for each point
        colors = ['blue'] * len(x)
        if len(colors) > 0:
            colors[-1] = 'red'  # Latest value
        if len(colors) > 1:
            colors[-11:-1] = ['green'] * min(10, len(colors)-1)  # Next past 10 days
        if len(colors) > 11:
            colors[-41:-11] = ['orange'] * min(30, len(colors)-11)  # Next past 30 days
        
        # Add scatter plot
        fig.add_trace(go.Scatter(
            x=x, 
            y=y, 
            mode='markers',
            marker=dict(color=colors, size=8),
            text=dates,  # This will be shown on hover
            hovertemplate='Date: %{text}<br>X: %{x}<br>Y: %{y}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'Butterfly Spread: {spread_name}',
            xaxis_title=f'{middle_contract} Implied Rate',
            yaxis_title='Butterfly Spread Value',
            showlegend=False
        )
        
        figs.append(fig)
    
    return figs

butterfly_scatterplots = create_butterfly_scatterplots(pivot, butterfly_spreads)

# Define chart titles and data
chart_data = [
    ('SOFR Implied Rates for Single Contracts', [col for col in pivot.columns if 'SR3' in col and '_' not in col]),
    ('SOFR Pack Rates', [col for col in pivot.columns if '_pack' in col]),
    ('SOFR Pack Differentials', [col for col in pivot.columns if '_diff' in col and '6m' not in col and 'spread' not in col]),
    ('SOFR 3-Month Calendar Spreads', [col for col in pivot.columns if '_spread' in col and '6m' not in col and '_diff' not in col]),
    ('SOFR 3-Month Calendar Spread Differentials', [col for col in pivot.columns if '_spread_' in col and '6m' not in col and '_diff' in col]),
    ('SOFR 6-Month Calendar Spreads', [col for col in pivot.columns if '6m_spread' in col and '6m_diff' not in col]),
    ('SOFR 6-Month Calendar Spread Differentials', [col for col in pivot.columns if '6m_diff' in col]),
    ('SOFR 6-month Butterfly Spreads', list(butterfly_spreads.keys()))
]


# Create Plotly figure
fig = make_subplots(rows=len(chart_data), cols=1, subplot_titles=[title for title, _ in chart_data])

for i, (title, cols) in enumerate(chart_data, start=1):
    for col in cols:
        fig.add_trace(
            go.Scatter(x=pivot.index, y=pivot[col], name=col, mode='lines'),
            row=i, col=1
        )
    fig.update_xaxes(title_text="Date", row=i, col=1)
    fig.update_yaxes(title_text="Rate (bps)", row=i, col=1)

fig.update_layout(height=400*len(chart_data), width=1200, title_text="SOFR Futures Analysis (Interactive)")

# Create Dash app for yield curve comparison
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("SOFR Yield Curve Comparison"),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=yield_curve_data.index.min(),
        max_date_allowed=yield_curve_data.index.max(),
        initial_visible_month=yield_curve_data.index.max(),
        start_date=yield_curve_data.index[-5],
        end_date=yield_curve_data.index[-1]
    ),
    html.Button('Update Graph', id='submit-button', n_clicks=0),
    dcc.Graph(id='yield-curve-graph')
])

@app.callback(
    Output('yield-curve-graph', 'figure'),
    Input('submit-button', 'n_clicks'),
    Input('date-picker-range', 'start_date'),
    Input('date-picker-range', 'end_date')
)
def update_graph(n_clicks, start_date, end_date):
    if n_clicks == 0:
        raise PreventUpdate

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (yield_curve_data.index >= start_date) & (yield_curve_data.index <= end_date)
    selected_data = yield_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=list(range(len(data))),
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis=dict(
            title='Contract Sequence',
            tickmode='array',
            tickvals=list(range(len(contract_cols))),
            ticktext=contract_cols,
            tickangle=45
        ),
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig

butterfly_scatterplots_html = "".join([fig.to_html(full_html=False, include_plotlyjs='cdn') for fig in butterfly_scatterplots])

# Generate HTML
combined_html = f"""
<html>
<head>
    <title>SOFR Futures Analysis</title>
</head>
<body>
    <h1>SOFR Futures Analysis</h1>
    <h2>Static Overview (High Resolution)</h2>
    {html_string}
    <h2>Interactive Charts</h2>
    {fig.to_html(full_html=False)}
    <h2>Butterfly Spread Scatterplots</h2>
    {butterfly_scatterplots_html}
    <h2>6-Month Butterfly Spread Curve</h2>
    <iframe src="http://127.0.0.1:8050/butterfly/" width="100%" height="800px" frameborder="0"></iframe>
    <h2>SOFR Yield Curve Comparison</h2>
    <iframe src="http://127.0.0.1:8050" width="100%" height="800px" frameborder="0"></iframe>
</body>
</html>
"""

# Save the combined HTML
with open(os.path.join(folder_path, 'sofr_analysis_dashboard_combined.html'), 'w') as f:
    f.write(combined_html)

print("Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'")

# Run both Dash apps
if __name__ == '__main__':
    server = Flask(__name__)
    app.server = server
    butterfly_app.server = server
    
    @server.route("/")
    def render_dashboard():
        return app.index()

    @server.route("/butterfly/")
    def render_butterfly():
        return butterfly_app.index()

    server.run(debug=True, port=8050)

In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import io
import base64
from dash import Dash, dcc, html, Input, Output, callback
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from flask import Flask

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month butterfly spreads
butterfly_spreads = {}
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    spread_name = f'{front}_{middle}_{back}_butterfly'
    butterfly_spreads[spread_name] = pivot[front] - (2 * pivot[middle]) + pivot[back]

# Add butterfly spreads to pivot table
for name, spread in butterfly_spreads.items():
    pivot[name] = spread
    
# Prepare data for butterfly spread curve
butterfly_curve_data = pd.DataFrame(index=pivot.index)
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    butterfly_curve_data[middle] = pivot[front] - (2 * pivot[middle]) + pivot[back]

# Remove rows with all NaN values
butterfly_curve_data = butterfly_curve_data.dropna(how='all')
    
print("Sample of butterfly_curve_data:")
print(butterfly_curve_data.head())  # Print the first 5 rows

print("\nShape of butterfly_curve_data:")
print(butterfly_curve_data.shape)

print("\nColumns of butterfly_curve_data:")
print(butterfly_curve_data.columns)

print("\nDescriptive statistics of butterfly_curve_data:")
print(butterfly_curve_data.describe())

print("\nSample values for a specific date:")
sample_date = butterfly_curve_data.index[0]  # Choose the first date
print(f"Values for {sample_date}:")
print(butterfly_curve_data.loc[sample_date])

print("\nRange of values:")
print(f"Min: {butterfly_curve_data.values.min()}")
print(f"Max: {butterfly_curve_data.values.max()}")

# If you want to see the raw calculation for a specific butterfly spread:
i = 0  # Choose the first butterfly spread
front = all_quarterly[i]
middle = all_quarterly[i+2]
back = all_quarterly[i+4]
print(f"\nRaw calculation for {front} - (2*{middle}) + {back}:")
print(f"{front}: {pivot[front].iloc[-1]}")
print(f"{middle}: {pivot[middle].iloc[-1]}")
print(f"{back}: {pivot[back].iloc[-1]}")
result = pivot[front].iloc[-1] - (2 * pivot[middle].iloc[-1]) + pivot[back].iloc[-1]
print(f"Result: {result}")
print(f"Stored value in butterfly_curve_data: {butterfly_curve_data[middle].iloc[-1]}")

# Create Dash app for butterfly spread curve
butterfly_app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

butterfly_app.layout = html.Div([
    html.H1("6-Month Butterfly Spread Curve"),
    dcc.DatePickerRange(
        id='butterfly-date-picker-range',
        min_date_allowed=butterfly_curve_data.index.min(),
        max_date_allowed=butterfly_curve_data.index.max(),
        initial_visible_month=butterfly_curve_data.index.max(),
        start_date=butterfly_curve_data.index[-5],
        end_date=butterfly_curve_data.index[-1]
    ),
    html.Button('Update Graph', id='butterfly-submit-button', n_clicks=0),
    dcc.Graph(id='butterfly-curve-graph')
])

@butterfly_app.callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (butterfly_curve_data.index >= start_date) & (butterfly_curve_data.index <= end_date)
    selected_data = butterfly_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        if not data.empty:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data,
                mode='lines+markers',
                name=date.strftime('%Y-%m-%d'),
                text=[f"{all_quarterly[i]} - (2*{col}) + {all_quarterly[i+4]}" for i, col in enumerate(data.index)],
                hovertemplate='%{text}<br>Value: %{y:.2f}bps<extra></extra>'
            ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Middle Contract',
            tickmode='array',
            tickvals=list(range(len(butterfly_curve_data.columns))),
            ticktext=butterfly_curve_data.columns,
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig


# Create scatterplots for butterfly spreads
def create_butterfly_scatterplots(pivot, butterfly_spreads):
    figs = []
    for spread_name, spread_values in butterfly_spreads.items():
        middle_contract = spread_name.split('_')[1]
        
        fig = go.Figure()
        
        x = pivot[middle_contract]
        y = spread_values
        
        # Sort the data by date
        sorted_indices = np.argsort(pivot.index)
        x = x.iloc[sorted_indices]
        y = y.iloc[sorted_indices]
        dates = pivot.index[sorted_indices]
        
        # Define color for each point
        colors = ['blue'] * len(x)
        if len(colors) > 0:
            colors[-1] = 'red'  # Latest value
        if len(colors) > 1:
            colors[-11:-1] = ['green'] * min(10, len(colors)-1)  # Next past 10 days
        if len(colors) > 11:
            colors[-41:-11] = ['orange'] * min(30, len(colors)-11)  # Next past 30 days
        
        # Add scatter plot
        fig.add_trace(go.Scatter(
            x=x, 
            y=y, 
            mode='markers',
            marker=dict(color=colors, size=8),
            text=dates,  # This will be shown on hover
            hovertemplate='Date: %{text}<br>X: %{x}<br>Y: %{y}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'Butterfly Spread: {spread_name}',
            xaxis_title=f'{middle_contract} Implied Rate',
            yaxis_title='Butterfly Spread Value',
            showlegend=False
        )
        
        figs.append(fig)
    
    return figs

butterfly_scatterplots = create_butterfly_scatterplots(pivot, butterfly_spreads)

# Define chart titles and data
chart_data = [
    ('SOFR Implied Rates for Single Contracts', [col for col in pivot.columns if 'SR3' in col and '_' not in col]),
    ('SOFR Pack Rates', [col for col in pivot.columns if '_pack' in col]),
    ('SOFR Pack Differentials', [col for col in pivot.columns if '_diff' in col and '6m' not in col and 'spread' not in col]),
    ('SOFR 3-Month Calendar Spreads', [col for col in pivot.columns if '_spread' in col and '6m' not in col and '_diff' not in col]),
    ('SOFR 3-Month Calendar Spread Differentials', [col for col in pivot.columns if '_spread_' in col and '6m' not in col and '_diff' in col]),
    ('SOFR 6-Month Calendar Spreads', [col for col in pivot.columns if '6m_spread' in col and '6m_diff' not in col]),
    ('SOFR 6-Month Calendar Spread Differentials', [col for col in pivot.columns if '6m_diff' in col]),
    ('SOFR 6-month Butterfly Spreads', list(butterfly_spreads.keys()))
]

# Create Plotly figure
fig = make_subplots(rows=len(chart_data), cols=1, subplot_titles=[title for title, _ in chart_data])

for i, (title, cols) in enumerate(chart_data, start=1):
    for col in cols:
        fig.add_trace(
            go.Scatter(x=pivot.index, y=pivot[col], name=col, mode='lines'),
            row=i, col=1
        )
    fig.update_xaxes(title_text="Date", row=i, col=1)
    fig.update_yaxes(title_text="Rate (bps)", row=i, col=1)

fig.update_layout(height=400*len(chart_data), width=1200, title_text="SOFR Futures Analysis (Interactive)")



app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("SOFR Analysis Dashboard"),
    
    dcc.Tabs([
        dcc.Tab(label='SOFR Yield Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='sofr-date-picker-range',
                    min_date_allowed=pivot.index.min(),
                    max_date_allowed=pivot.index.max(),
                    initial_visible_month=pivot.index.max(),
                    start_date=pivot.index[-5],
                    end_date=pivot.index[-1]
                ),
                html.Button('Update SOFR Graph', id='sofr-submit-button', n_clicks=0),
                dcc.Graph(id='sofr-yield-curve-graph')
            ])
        ]),
        dcc.Tab(label='6-Month Butterfly Spread Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='butterfly-date-picker-range',
                    min_date_allowed=butterfly_curve_data.index.min(),
                    max_date_allowed=butterfly_curve_data.index.max(),
                    initial_visible_month=butterfly_curve_data.index.max(),
                    start_date=butterfly_curve_data.index[-5],
                    end_date=butterfly_curve_data.index[-1]
                ),
                html.Button('Update Butterfly Graph', id='butterfly-submit-button', n_clicks=0),
                dcc.Graph(id='butterfly-curve-graph')
            ])
        ]),
    ])
])

@app.callback(
    Output('sofr-yield-curve-graph', 'figure'),
    Input('sofr-submit-button', 'n_clicks'),
    Input('sofr-date-picker-range', 'start_date'),
    Input('sofr-date-picker-range', 'end_date')
)
def update_sofr_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (pivot.index >= start_date) & (pivot.index <= end_date)
    selected_data = pivot.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=data.index,
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            hovertemplate='%{x}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis_title='Contract',
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig

@app.callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (butterfly_curve_data.index >= start_date) & (butterfly_curve_data.index <= end_date)
    selected_data = butterfly_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        if not data.empty:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data,
                mode='lines+markers',
                name=date.strftime('%Y-%m-%d'),
                text=[f"{all_quarterly[i]} - (2*{col}) + {all_quarterly[i+4]}" for i, col in enumerate(data.index)],
                hovertemplate='%{text}<br>Value: %{y:.2f}bps<extra></extra>'
            ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Middle Contract',
            tickmode='array',
            tickvals=list(range(len(butterfly_curve_data.columns))),
            ticktext=butterfly_curve_data.columns,
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

In [1]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month butterfly spreads
butterfly_curve_data = pd.DataFrame(index=pivot.index)
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    butterfly_curve_data[middle] = pivot[front] - (2 * pivot[middle]) + pivot[back]

# Remove rows with all NaN values
butterfly_curve_data = butterfly_curve_data.dropna(how='all')

# Create Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("SOFR Analysis Dashboard"),
    
    dcc.Tabs([
        dcc.Tab(label='SOFR Yield Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='sofr-date-picker-range',
                    min_date_allowed=pivot.index.min(),
                    max_date_allowed=pivot.index.max(),
                    initial_visible_month=pivot.index.max(),
                    start_date=pivot.index[-5],
                    end_date=pivot.index[-1]
                ),
                html.Button('Update SOFR Graph', id='sofr-submit-button', n_clicks=0),
                dcc.Graph(id='sofr-yield-curve-graph')
            ])
        ]),
        dcc.Tab(label='6-Month Butterfly Spread Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='butterfly-date-picker-range',
                    min_date_allowed=butterfly_curve_data.index.min(),
                    max_date_allowed=butterfly_curve_data.index.max(),
                    initial_visible_month=butterfly_curve_data.index.max(),
                    start_date=butterfly_curve_data.index[-5],
                    end_date=butterfly_curve_data.index[-1]
                ),
                html.Button('Update Butterfly Graph', id='butterfly-submit-button', n_clicks=0),
                dcc.Graph(id='butterfly-curve-graph')
            ])
        ]),
    ])
])

@app.callback(
    Output('sofr-yield-curve-graph', 'figure'),
    Input('sofr-submit-button', 'n_clicks'),
    Input('sofr-date-picker-range', 'start_date'),
    Input('sofr-date-picker-range', 'end_date')
)
def update_sofr_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (pivot.index >= start_date) & (pivot.index <= end_date)
    selected_data = pivot.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=data.index,
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            hovertemplate='%{x}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis_title='Contract',
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig

@app.callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (butterfly_curve_data.index >= start_date) & (butterfly_curve_data.index <= end_date)
    selected_data = butterfly_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        if not data.empty:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data,
                mode='lines+markers',
                name=date.strftime('%Y-%m-%d'),
                text=[f"{all_quarterly[i]} - (2*{col}) + {all_quarterly[i+4]}" for i, col in enumerate(data.index)],
                hovertemplate='%{text}<br>Value: %{y:.2f}bps<extra></extra>'
            ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Middle Contract',
            tickmode='array',
            tickvals=list(range(len(butterfly_curve_data.columns))),
            ticktext=butterfly_curve_data.columns,
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

In [3]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month butterfly spreads
butterfly_curve_data = pd.DataFrame(index=pivot.index)
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    butterfly_curve_data[middle] = pivot[front] - (2 * pivot[middle]) + pivot[back]

# Remove rows with all NaN values
butterfly_curve_data = butterfly_curve_data.dropna(how='all')

# Prepare yield curve data
def prepare_yield_curve_data(pivot):
    contract_cols = [col for col in pivot.columns if 'SR3' in col and '_' not in col]
    contract_cols.sort(key=lambda x: (x[-4:], {'H': 1, 'M': 2, 'U': 3, 'Z': 4}[x[-5]]))
    yield_curve_data = pivot[contract_cols]
    return yield_curve_data, contract_cols

yield_curve_data, contract_cols = prepare_yield_curve_data(pivot)

# Create Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("SOFR Analysis Dashboard"),
    
    dcc.Tabs([
        dcc.Tab(label='SOFR Yield Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='sofr-date-picker-range',
                    min_date_allowed=yield_curve_data.index.min(),
                    max_date_allowed=yield_curve_data.index.max(),
                    initial_visible_month=yield_curve_data.index.max(),
                    start_date=yield_curve_data.index[-5],
                    end_date=yield_curve_data.index[-1]
                ),
                html.Button('Update SOFR Graph', id='sofr-submit-button', n_clicks=0),
                dcc.Graph(id='sofr-yield-curve-graph')
            ])
        ]),
        dcc.Tab(label='6-Month Butterfly Spread Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='butterfly-date-picker-range',
                    min_date_allowed=butterfly_curve_data.index.min(),
                    max_date_allowed=butterfly_curve_data.index.max(),
                    initial_visible_month=butterfly_curve_data.index.max(),
                    start_date=butterfly_curve_data.index[-5],
                    end_date=butterfly_curve_data.index[-1]
                ),
                html.Button('Update Butterfly Graph', id='butterfly-submit-button', n_clicks=0),
                dcc.Graph(id='butterfly-curve-graph')
            ])
        ]),
    ])
])

@app.callback(
    Output('sofr-yield-curve-graph', 'figure'),
    Input('sofr-submit-button', 'n_clicks'),
    Input('sofr-date-picker-range', 'start_date'),
    Input('sofr-date-picker-range', 'end_date')
)
def update_sofr_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (yield_curve_data.index >= start_date) & (yield_curve_data.index <= end_date)
    selected_data = yield_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=list(range(len(data))),
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis=dict(
            title='Contract Sequence',
            tickmode='array',
            tickvals=list(range(len(contract_cols))),
            ticktext=contract_cols,
            tickangle=45
        ),
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig

@app.callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (butterfly_curve_data.index >= start_date) & (butterfly_curve_data.index <= end_date)
    selected_data = butterfly_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        if not data.empty:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data,
                mode='lines+markers',
                name=date.strftime('%Y-%m-%d'),
                text=[f"{all_quarterly[i]} - (2*{col}) + {all_quarterly[i+4]}" for i, col in enumerate(data.index)],
                hovertemplate='%{text}<br>Value: %{y:.2f}bps<extra></extra>'
            ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Middle Contract',
            tickmode='array',
            tickvals=list(range(len(butterfly_curve_data.columns))),
            ticktext=butterfly_curve_data.columns,
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

In [6]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month butterfly spreads
butterfly_curve_data = pd.DataFrame(index=pivot.index)
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    butterfly_curve_data[middle] = pivot[front] - (2 * pivot[middle]) + pivot[back]

# Remove rows with all NaN values
butterfly_curve_data = butterfly_curve_data.dropna(how='all')

# Prepare yield curve data
def prepare_yield_curve_data(pivot):
    contract_cols = [col for col in pivot.columns if 'SR3' in col and '_' not in col]
    contract_cols.sort(key=lambda x: (x[-4:], {'H': 1, 'M': 2, 'U': 3, 'Z': 4}[x[-5]]))
    yield_curve_data = pivot[contract_cols]
    return yield_curve_data, contract_cols

yield_curve_data, contract_cols = prepare_yield_curve_data(pivot)

# Create Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("SOFR Analysis Dashboard"),
    
    dcc.Tabs([
        dcc.Tab(label='SOFR Yield Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='sofr-date-picker-range',
                    min_date_allowed=yield_curve_data.index.min(),
                    max_date_allowed=yield_curve_data.index.max(),
                    initial_visible_month=yield_curve_data.index.max(),
                    start_date=yield_curve_data.index[-5],
                    end_date=yield_curve_data.index[-1]
                ),
                html.Button('Update SOFR Graph', id='sofr-submit-button', n_clicks=0),
                dcc.Graph(id='sofr-yield-curve-graph')
            ])
        ]),
        dcc.Tab(label='6-Month Butterfly Spread Curve', children=[
            html.Div([
                dcc.DatePickerRange(
                    id='butterfly-date-picker-range',
                    min_date_allowed=butterfly_curve_data.index.min(),
                    max_date_allowed=butterfly_curve_data.index.max(),
                    initial_visible_month=butterfly_curve_data.index.max(),
                    start_date=butterfly_curve_data.index[-5],
                    end_date=butterfly_curve_data.index[-1]
                ),
                html.Button('Update Butterfly Graph', id='butterfly-submit-button', n_clicks=0),
                dcc.Graph(id='butterfly-curve-graph')
            ])
        ]),
    ])
])

@app.callback(
    Output('sofr-yield-curve-graph', 'figure'),
    Input('sofr-submit-button', 'n_clicks'),
    Input('sofr-date-picker-range', 'start_date'),
    Input('sofr-date-picker-range', 'end_date')
)
def update_sofr_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (yield_curve_data.index >= start_date) & (yield_curve_data.index <= end_date)
    selected_data = yield_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=list(range(len(data))),
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis=dict(
            title='Contract Sequence',
            tickmode='array',
            tickvals=list(range(len(contract_cols))),
            ticktext=contract_cols,
            tickangle=45
        ),
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig

@app.callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (butterfly_curve_data.index >= start_date) & (butterfly_curve_data.index <= end_date)
    selected_data = butterfly_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        if not data.empty:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data,
                mode='lines+markers',
                name=date.strftime('%Y-%m-%d'),
                text=[f"{all_quarterly[i]} - (2*{col}) + {all_quarterly[i+4]}" for i, col in enumerate(data.index)],
                hovertemplate='%{text}<br>Value: %{y:.2f}bps<extra></extra>'
            ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Middle Contract',
            tickmode='array',
            tickvals=list(range(len(butterfly_curve_data.columns))),
            ticktext=butterfly_curve_data.columns,
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig

# Define chart titles and data
chart_data = [
    ('SOFR Implied Rates for Single Contracts', [col for col in pivot.columns if 'SR3' in col and '_' not in col]),
    ('SOFR Pack Rates', [col for col in pivot.columns if '_pack' in col]),
    ('SOFR Pack Differentials', [col for col in pivot.columns if '_diff' in col and '6m' not in col and 'spread' not in col]),
    ('SOFR 3-Month Calendar Spreads', [col for col in pivot.columns if '_spread' in col and '6m' not in col and '_diff' not in col]),
    ('SOFR 3-Month Calendar Spread Differentials', [col for col in pivot.columns if '_spread_' in col and '6m' not in col and '_diff' in col]),
    ('SOFR 6-Month Calendar Spreads', [col for col in pivot.columns if '6m_spread' in col and '6m_diff' not in col]),
    ('SOFR 6-Month Calendar Spread Differentials', [col for col in pivot.columns if '6m_diff' in col]),
    ('SOFR 6-month Butterfly Spreads', list(butterfly_curve_data.columns))
]

# Create Plotly figure for the main interactive chart
fig = make_subplots(rows=len(chart_data), cols=1, subplot_titles=[title for title, _ in chart_data])

for i, (title, cols) in enumerate(chart_data, start=1):
    for col in cols:
        fig.add_trace(
            go.Scatter(x=pivot.index, y=pivot[col], name=col, mode='lines'),
            row=i, col=1
        )
    fig.update_xaxes(title_text="Date", row=i, col=1)
    fig.update_yaxes(title_text="Rate (bps)", row=i, col=1)

fig.update_layout(height=400*len(chart_data), width=1200, title_text="SOFR Futures Analysis (Interactive)")

# Generate HTML
combined_html = f"""
<html>
<head>
    <title>SOFR Futures Analysis</title>
</head>
<body>
    <h1>SOFR Futures Analysis</h1>
    <h2>Interactive Charts</h2>
    {fig.to_html(full_html=False)}
    <h2>6-Month Butterfly Spread Curve</h2>
    <iframe src="http://127.0.0.1:8050/butterfly/" width="100%" height="800px" frameborder="0"></iframe>
    <h2>SOFR Yield Curve Comparison</h2>
    <iframe src="http://127.0.0.1:8050" width="100%" height="800px" frameborder="0"></iframe>
</body>
</html>
"""

# Save the combined HTML
with open(os.path.join(folder_path, 'sofr_analysis_dashboard_combined.html'), 'w') as f:
    f.write(combined_html)

print("Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'")

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'


In [10]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

# Set the specific file path
folder_path = r"C:\Users\theon\OneDrive\Documents\substack\MacroTradingToolkit\SOFR 3"

def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['contract'] = os.path.splitext(os.path.basename(file_path))[0].split(',')[0].replace('CME_', '')
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['implied_rate'] = (100 - df['close']) * 100  # Convert to basis points
    return df[['date', 'contract', 'implied_rate']]

# Read all CSV files in the SOFR 3 folder
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename != 'sofr_futures_analysis_tradingview.csv':
        file_path = os.path.join(folder_path, filename)
        all_data.append(process_csv(file_path))

# Combine all data
combined_data = pd.concat(all_data)
combined_data = combined_data.sort_values(['date', 'contract'])
combined_data = combined_data.groupby(['date', 'contract'], as_index=False).last()

# Create a pivot table
pivot = combined_data.pivot(index='date', columns='contract', values='implied_rate')
pivot = pivot.sort_index(axis=1)

def get_next_n_quarterly(contracts, n):
    quarterly_months = ['H', 'M', 'U', 'Z']
    sorted_contracts = sorted(contracts, key=lambda x: (x[-4:], quarterly_months.index(x[-5])))
    result = [contract for contract in sorted_contracts if contract[-5] in quarterly_months][:n]
    return result

# Calculate SOFR packs
packs = ['white', 'red', 'green', 'blue', 'gold']
all_quarterly = get_next_n_quarterly(pivot.columns, 20)

for i, pack in enumerate(packs):
    pack_contracts = all_quarterly[i*4:(i+1)*4]
    if pack_contracts:
        pivot[f'{pack}_pack'] = pivot[pack_contracts].mean(axis=1)

# Calculate pack differentials
for i in range(len(packs)-1):
    pivot[f'{packs[i]}_{packs[i+1]}_diff'] = pivot[f'{packs[i]}_pack'] - pivot[f'{packs[i+1]}_pack']

# Calculate 3-month calendar spreads
for i in range(len(all_quarterly) - 1):
    front = all_quarterly[i]
    back = all_quarterly[i+1]
    pivot[f'{front}_{back}_spread'] = pivot[front] - pivot[back]

# Calculate 3-month calendar spread differentials
for i in range(len(all_quarterly) - 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+1]}_spread'
    spread2 = f'{all_quarterly[i+1]}_{all_quarterly[i+2]}_spread'
    pivot[f'{spread1}_{spread2}_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month calendar spreads
for i in range(0, len(all_quarterly) - 2, 2):
    front = all_quarterly[i]
    back = all_quarterly[i+2]
    pivot[f'{front}_{back}_6m_spread'] = pivot[front] - pivot[back]

# Calculate 6-month calendar spread differentials
for i in range(0, len(all_quarterly) - 4, 2):
    spread1 = f'{all_quarterly[i]}_{all_quarterly[i+2]}_6m_spread'
    spread2 = f'{all_quarterly[i+2]}_{all_quarterly[i+4]}_6m_spread'
    pivot[f'{spread1}_{spread2}_6m_diff'] = pivot[spread1] - pivot[spread2]

# Calculate 6-month butterfly spreads
butterfly_spreads = {}
for i in range(len(all_quarterly) - 4):
    front = all_quarterly[i]
    middle = all_quarterly[i+2]
    back = all_quarterly[i+4]
    spread_name = f'{front}_{middle}_{back}_butterfly'
    butterfly_spreads[spread_name] = (pivot[front] + pivot[back]) - (2 * pivot[middle])

# Add butterfly spreads to pivot table
for name, spread in butterfly_spreads.items():
    pivot[name] = spread

# Prepare yield curve data
def prepare_yield_curve_data(pivot):
    contract_cols = [col for col in pivot.columns if 'SR3' in col and '_' not in col]
    contract_cols.sort(key=lambda x: (x[-4:], {'H': 1, 'M': 2, 'U': 3, 'Z': 4}[x[-5]]))
    yield_curve_data = pivot[contract_cols]
    return yield_curve_data, contract_cols

yield_curve_data, contract_cols = prepare_yield_curve_data(pivot)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("SOFR Analysis Dashboard"),
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/yield-curve':
        return html.Div([
            dcc.DatePickerRange(
                id='sofr-date-picker-range',
                min_date_allowed=yield_curve_data.index.min(),
                max_date_allowed=yield_curve_data.index.max(),
                initial_visible_month=yield_curve_data.index.max(),
                start_date=yield_curve_data.index[-5],
                end_date=yield_curve_data.index[-1]
            ),
            html.Button('Update SOFR Graph', id='sofr-submit-button', n_clicks=0),
            dcc.Graph(id='sofr-yield-curve-graph')
        ])
    elif pathname == '/butterfly':
        return html.Div([
            dcc.DatePickerRange(
                id='butterfly-date-picker-range',
                min_date_allowed=pivot.index.min(),
                max_date_allowed=pivot.index.max(),
                initial_visible_month=pivot.index.max(),
                start_date=pivot.index[-5],
                end_date=pivot.index[-1]
            ),
            html.Button('Update Butterfly Graph', id='butterfly-submit-button', n_clicks=0),
            dcc.Graph(id='butterfly-curve-graph')
        ])
    else:
        return html.Div([
            html.H3('Please select a chart:'),
            dcc.Link('SOFR Yield Curve', href='/yield-curve'),
            html.Br(),
            dcc.Link('6-Month Butterfly Spread Curve', href='/butterfly')
        ])
    
@app.callback(
    Output('sofr-yield-curve-graph', 'figure'),
    Input('sofr-submit-button', 'n_clicks'),
    Input('sofr-date-picker-range', 'start_date'),
    Input('sofr-date-picker-range', 'end_date')
)
def update_sofr_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (yield_curve_data.index >= start_date) & (yield_curve_data.index <= end_date)
    selected_data = yield_curve_data.loc[mask]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        fig.add_trace(go.Scatter(
            x=list(range(len(data))),
            y=data,
            mode='lines+markers',
            name=date.strftime('%Y-%m-%d'),
            text=data.index,
            hovertemplate='%{text}: %{y:.2f}bps<extra></extra>'
        ))

    fig.update_layout(
        title='SOFR Yield Curve Comparison',
        xaxis=dict(
            title='Contract Sequence',
            tickmode='array',
            tickvals=list(range(len(contract_cols))),
            ticktext=contract_cols,
            tickangle=45
        ),
        yaxis_title='Implied SOFR Rate (bps)',
        hovermode='x unified'
    )

    return fig

@app.callback(
    Output('butterfly-curve-graph', 'figure'),
    Input('butterfly-submit-button', 'n_clicks'),
    Input('butterfly-date-picker-range', 'start_date'),
    Input('butterfly-date-picker-range', 'end_date')
)
def update_butterfly_graph(n_clicks, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    mask = (pivot.index >= start_date) & (pivot.index <= end_date)
    selected_data = pivot.loc[mask, [col for col in pivot.columns if 'butterfly' in col]]

    fig = go.Figure()

    for date in selected_data.index:
        data = selected_data.loc[date].dropna()
        if not data.empty:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data,
                mode='lines+markers',
                name=date.strftime('%Y-%m-%d'),
                text=[f"({name.split('_')[0]} + {name.split('_')[2]}) - 2*{name.split('_')[1]}" for name in data.index],
                hovertemplate='%{text}<br>Value: %{y:.2f}bps<extra></extra>'
            ))

    fig.update_layout(
        title='6-Month Butterfly Spread Curve',
        xaxis=dict(
            title='Butterfly Spread',
            tickmode='array',
            tickvals=list(range(len(selected_data.columns))),
            ticktext=[col.split('_')[1] for col in selected_data.columns],
            tickangle=45
        ),
        yaxis_title='Butterfly Spread Value (bps)',
        hovermode='x unified'
    )

    return fig

# Create scatterplots for butterfly spreads
def create_butterfly_scatterplots(pivot, butterfly_spreads):
    figs = []
    for spread_name, spread_values in butterfly_spreads.items():
        middle_contract = spread_name.split('_')[1]
        
        fig = go.Figure()
        
        x = pivot[middle_contract]
        y = spread_values
        
        # Sort the data by date
        sorted_indices = np.argsort(pivot.index)
        x = x.iloc[sorted_indices]
        y = y.iloc[sorted_indices]
        dates = pivot.index[sorted_indices]
        
        # Define color for each point
        colors = ['blue'] * len(x)
        if len(colors) > 0:
            colors[-1] = 'red'  # Latest value
        if len(colors) > 1:
            colors[-11:-1] = ['green'] * min(10, len(colors)-1)  # Next past 10 days
        if len(colors) > 11:
            colors[-41:-11] = ['orange'] * min(30, len(colors)-11)  # Next past 30 days
        
        # Add scatter plot
        fig.add_trace(go.Scatter(
            x=x, 
            y=y, 
            mode='markers',
            marker=dict(color=colors, size=8),
            text=dates,  # This will be shown on hover
            hovertemplate='Date: %{text}<br>X: %{x}<br>Y: %{y}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'Butterfly Spread: {spread_name}',
            xaxis_title=f'{middle_contract} Implied Rate',
            yaxis_title='Butterfly Spread Value',
            showlegend=False
        )
        
        figs.append(fig)
    
    return figs

butterfly_scatterplots = create_butterfly_scatterplots(pivot, butterfly_spreads)

# Define chart titles and data
chart_data = [
    ('SOFR Implied Rates for Single Contracts', [col for col in pivot.columns if 'SR3' in col and '_' not in col]),
    ('SOFR Pack Rates', [col for col in pivot.columns if '_pack' in col]),
    ('SOFR Pack Differentials', [col for col in pivot.columns if '_diff' in col and '6m' not in col and 'spread' not in col]),
    ('SOFR 3-Month Calendar Spreads', [col for col in pivot.columns if '_spread' in col and '6m' not in col and '_diff' not in col]),
    ('SOFR 3-Month Calendar Spread Differentials', [col for col in pivot.columns if '_spread_' in col and '6m' not in col and '_diff' in col]),
    ('SOFR 6-Month Calendar Spreads', [col for col in pivot.columns if '6m_spread' in col and '6m_diff' not in col]),
    ('SOFR 6-Month Calendar Spread Differentials', [col for col in pivot.columns if '6m_diff' in col]),
    ('SOFR 6-month Butterfly Spreads', list(butterfly_spreads.keys()))
]

# Create Plotly figure for the main interactive chart
fig = make_subplots(rows=len(chart_data), cols=1, subplot_titles=[title for title, _ in chart_data])

for i, (title, cols) in enumerate(chart_data, start=1):
    for col in cols:
        fig.add_trace(
            go.Scatter(x=pivot.index, y=pivot[col], name=col, mode='lines'),
            row=i, col=1
        )
    fig.update_xaxes(title_text="Date", row=i, col=1)
    fig.update_yaxes(title_text="Rate (bps)", row=i, col=1)

fig.update_layout(height=400*len(chart_data), width=1200, title_text="SOFR Futures Analysis (Interactive)")

butterfly_scatterplots_html = "".join([fig.to_html(full_html=False, include_plotlyjs='cdn') for fig in butterfly_scatterplots])

# Generate HTML
combined_html = f"""
<html>
<head>
    <title>SOFR Futures Analysis</title>
</head>
<body>
    <h1>SOFR Futures Analysis</h1>
    <h2>Interactive Charts</h2>
    {fig.to_html(full_html=False)}
    <h2>Butterfly Spread Scatterplots</h2>
    {butterfly_scatterplots_html}
    <h2>SOFR Yield Curve Comparison</h2>
    <iframe src="http://127.0.0.1:8050/yield-curve" width="100%" height="800px" frameborder="0"></iframe>
    <h2>6-Month Butterfly Spread Curve</h2>
    <iframe src="http://127.0.0.1:8050/butterfly" width="100%" height="800px" frameborder="0"></iframe>
</body>
</html>
"""

# Save the combined HTML
with open(os.path.join(folder_path, 'sofr_analysis_dashboard_combined.html'), 'w') as f:
    f.write(combined_html)

print("Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'")

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

Analysis complete. Dashboard saved as 'sofr_analysis_dashboard_combined.html'
